In [2]:
import os
from PIL import Image
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
from utils import datasets
from utils import utils
from utils import train_val
from utils import net #网络文件于此
import warnings

# 完全禁用警告
warnings.filterwarnings("ignore")

In [3]:
from utils.utils import Config,Logs,BestSelector
config=utils.Config(
    dataset_sep=[
        0.82,0.17,0.01          
        ],
    resize_size=420,
    batch_size=32,
    lr=0.0007,
    epochs=30,
    hidden_size=256,
    optim="Adam",
    momentum=0.9,
    weight_decay=1e-4,
    seed=42,
    mean= [0.50638 ,0.49962538 ,0.45205265],
    std=[0.23568255 ,0.24141274 ,0.25167742],
    source_dir=r"Cifar-10",#原始数据集，每个分类一个文件夹，每个文件夹里包含多个图片
    data_path=r"data\Cifar-10",#项目数据集
    # classes=["Apple","Carambola","Pear","Plum","Tomatoes"],
    device="cuda" if torch.cuda.is_available() else "cpu",
)
print(config)

dataset_sep : [0.82, 0.17, 0.01] 
resize_size : 420 
batch_size : 32 
lr : 0.0007 
epochs : 30 
hidden_size : 256 
optim : Adam 
momentum : 0.9 
weight_decay : 0.0001 
seed : 42 
mean : [0.50638, 0.49962538, 0.45205265] 
std : [0.23568255, 0.24141274, 0.25167742] 
source_dir : Cifar-10 
data_path : data\Cifar-10 
device : cuda 


In [4]:
from tqdm import tqdm

#设置随机数种子
torch.manual_seed(config.seed)

isSplit= (not datasets.check_data_exist(config.data_path))#数据集不存在，则从原始数据存放处，转移数据集
if isSplit:
    print(f"{config.data_path} 数据集不存在，将从source_dir:{config.source_dir}中获取数据")
    print(f"清理源文件夹:{config.data_path}")
    datasets.clear_folder(config.data_path)
    
    for dir in tqdm(config.classes,desc="处理原始数据："):
        source_dir=os.path.join(config.source_dir, dir)
        print(source_dir)
        datasets.split_data(source_dir,target_dir=config.data_path,label=dir,sep=config.dataset_sep)
else:
    print(f"{config.data_path} 数据集已存在，无需重新划分")



data\Cifar-10 数据集已存在，无需重新划分


In [5]:
'''
def get_data_path(typ,path=config.data_path):
    return os.path.join(path,typ)

train_dataset=datasets.CustomImageDataset(
    get_data_path("train"),
    classes=config.classes,
    transform=datasets.get_transform(
        resize_size=config.resize_size,
        mean=config.mean,std=config.std
        )
)

val_dataset=datasets.CustomImageDataset(
    get_data_path("val"),
    classes=config.classes,
    transform=datasets.get_transform(
        "val",
        config.resize_size,
        
        mean=config.mean,
        std=config.std
        )
)
'''


'\ndef get_data_path(typ,path=config.data_path):\n    return os.path.join(path,typ)\n\ntrain_dataset=datasets.CustomImageDataset(\n    get_data_path("train"),\n    classes=config.classes,\n    transform=datasets.get_transform(\n        resize_size=config.resize_size,\n        mean=config.mean,std=config.std\n        )\n)\n\nval_dataset=datasets.CustomImageDataset(\n    get_data_path("val"),\n    classes=config.classes,\n    transform=datasets.get_transform(\n        "val",\n        config.resize_size,\n        \n        mean=config.mean,\n        std=config.std\n        )\n)\n'

#### 加载数据集，可本地数据集。调用datasets.CustomImageDataset

In [6]:
#加载cifar-10
import torchvision
train_val_dataset = torchvision.datasets.CIFAR10(
    root='./data/Cifar-10', 
    train=True, 
    download=True, 
    transform=datasets.get_transform(
        chance="train",
        resize_size=64,
        mean=(0.5, 0.5, 0.5), 
        std=(0.5, 0.5, 0.5)
    )
)
train_dataset,val_dataset=datasets.get_parts_of_datasets(train_val_dataset,rate=0.75,only_train=False)#训练验证3-1开

test_dataset = torchvision.datasets.CIFAR10(root='./data/Cifar-10', train=False, download=True, transform=datasets.get_transform(
    chance="val",
    resize_size=64,
    mean=(0.5, 0.5, 0.5), 
    std=(0.5, 0.5, 0.5)
))
print(f"原始训练集大小：{len(train_dataset)}")
print(f"原始验证集大小：{len(val_dataset)}")
print(f"原始测试集大小：{len(test_dataset)}")

Files already downloaded and verified
Files already downloaded and verified
原始训练集大小：37500
原始验证集大小：12500
原始测试集大小：10000


#### load数据集，并根据数据量进行裁剪

In [7]:
crop_rate=0.25
train_dataset_crop=datasets.get_parts_of_datasets(train_dataset,crop_rate)
val_dataset_crop=datasets.get_parts_of_datasets(val_dataset,crop_rate)
test_dataset_crop=datasets.get_parts_of_datasets(test_dataset,crop_rate)
train_dataset_size=len(train_dataset_crop)
val_dataset_size=len(val_dataset_crop)
test_dataset_size=len(test_dataset_crop)

print(f"本次训练用训练集大小：{len(train_dataset_crop)}")
print(f"本次训练用验证集大小：{len(val_dataset_crop)}")
print(f"本次测试用测试集大小：{len(test_dataset_crop)}")

train_loader=DataLoader(train_dataset_crop,batch_size=config.batch_size,shuffle=True,drop_last=True)
val_loader=DataLoader(val_dataset_crop,batch_size=config.batch_size,shuffle=False)
test_loader=DataLoader(test_dataset_crop,batch_size=config.batch_size,shuffle=False)

print(f"tarin_dataloader加载完毕, {len(train_loader)}个batch, batch大小为{config.batch_size}")
print(f"val_dataloader  加载完毕, {len(val_loader)}个batch, batch大小为{config.batch_size}")
print(f"test_dataloader 加载完毕, {len(test_loader)}个batch, batch大小为{config.batch_size}")


本次训练用训练集大小：9375
本次训练用验证集大小：3125
本次测试用测试集大小：2500
tarin_dataloader加载完毕, 292个batch, batch大小为32
val_dataloader  加载完毕, 98个batch, batch大小为32
test_dataloader 加载完毕, 79个batch, batch大小为32


#### 记录本次训练和测试用的数据量

In [8]:
config.update(
    train_datasize=train_dataset_size,
    val_datasetsize=val_dataset_size,
    test_datasetsize=test_dataset_size,
    epochs=20,
    classes=train_val_dataset.classes,#原始数据集保留classes
)
config

dataset_sep : [0.82, 0.17, 0.01] 
resize_size : 420 
batch_size : 32 
lr : 0.0007 
epochs : 20 
hidden_size : 256 
optim : Adam 
momentum : 0.9 
weight_decay : 0.0001 
seed : 42 
mean : [0.50638, 0.49962538, 0.45205265] 
std : [0.23568255, 0.24141274, 0.25167742] 
source_dir : Cifar-10 
data_path : data\Cifar-10 
device : cuda 
train_datasize : 9375 
val_datasetsize : 3125 
test_datasetsize : 2500 
classes : ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 

#### 加载预训练模型

In [9]:

def get_pretrained(hidden_size=config.hidden_size,config=config):
    '''
        获取预训练模型
        @param hidden_size: 隐藏层大小
        @param config: 配置文件
        @return: 预训练模型
    '''
    model=net.BinaryClassificationMobileNetV3Large(out_size=len(config.classes))
    return model.to(config.device)



In [10]:
#查看数据加载情况
for inputs, labels in train_loader:
    inputs=inputs.to(config.device)
    labels=labels.to(config.device)
    print(inputs.shape)
    print(labels.shape)
    config.update(inputs_shape=inputs.shape)
    break
config

torch.Size([32, 3, 224, 224])
torch.Size([32])


dataset_sep : [0.82, 0.17, 0.01] 
resize_size : 420 
batch_size : 32 
lr : 0.0007 
epochs : 20 
hidden_size : 256 
optim : Adam 
momentum : 0.9 
weight_decay : 0.0001 
seed : 42 
mean : [0.50638, 0.49962538, 0.45205265] 
std : [0.23568255, 0.24141274, 0.25167742] 
source_dir : Cifar-10 
data_path : data\Cifar-10 
device : cuda 
train_datasize : 9375 
val_datasetsize : 3125 
test_datasetsize : 2500 
classes : ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 
inputs_shape : torch.Size([32, 3, 224, 224]) 

In [11]:

bestMod=utils.BestSelector(acc=0)
train_logs=utils.Logs()
model=get_pretrained()
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss()
if config.optim == "Adam":
    optimizer = optim.Adam(model.parameters(), lr=config.lr)
else:
    optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=0.9, weight_decay=1e-4)
    
bestMod,train_logs=train_val.train_model(
            rmodel, 
            criterion,
            optimizer, 
            train_loader,
            val_loader,
            bestMod=bestMod,
            train_logs=train_logs,
            config=config, 
            # num_epochs=config.epochs
        )



Training Epochs:   5%|▌         | 1/20 [00:43<13:39, 43.13s/it]

Epoch 1/20, Loss: 1.9470,  Val Accuracy: 0.5136, AP: 0.0918, Precision: 0.5088, Recall: 0.5121
当前最好的模型： acc : 0.5136 ,loss : 1.9469972971367508 ,precision : 0.5087996884936903 ,recall : 0.5120777275448403 ,ap : 0.09176705607652239 ,epoch : 0 


Training Epochs:  10%|█         | 2/20 [01:23<12:28, 41.60s/it]

Epoch 2/20, Loss: 1.8364,  Val Accuracy: 0.5853, AP: 0.0795, Precision: 0.6353, Recall: 0.5832
当前最好的模型： acc : 0.58528 ,loss : 1.8363701737906835 ,precision : 0.6352891260010255 ,recall : 0.5832391450665615 ,ap : 0.07954825423814013 ,epoch : 1 


Training Epochs:  15%|█▌        | 3/20 [02:07<12:08, 42.83s/it]

Epoch 3/20, Loss: 1.8058,  Val Accuracy: 0.6310, AP: 0.0957, Precision: 0.6651, Recall: 0.6312
当前最好的模型： acc : 0.63104 ,loss : 1.8058339798287169 ,precision : 0.6651199249434854 ,recall : 0.631191739626835 ,ap : 0.09567778772655369 ,epoch : 2 


Training Epochs:  20%|██        | 4/20 [02:48<11:11, 41.98s/it]

Epoch 4/20, Loss: 1.7944,  Val Accuracy: 0.6362, AP: 0.0945, Precision: 0.6580, Recall: 0.6358
当前最好的模型： acc : 0.63616 ,loss : 1.7944207358850193 ,precision : 0.6579830670227833 ,recall : 0.6358197459948413 ,ap : 0.0944935170152956 ,epoch : 3 


Training Epochs:  20%|██        | 4/20 [02:56<11:44, 44.01s/it]


KeyboardInterrupt: 

In [13]:
print(f"{config.epochs} epoch中 最好的模型")
print(bestMod)

20 epoch中 最好的模型
acc : 0.63616 ,loss : 1.7944207358850193 ,precision : 0.6579830670227833 ,recall : 0.6358197459948413 ,ap : 0.0944935170152956 ,epoch : 3 


In [14]:
saveDir=r'save_weights'
utils.saveProcess(
    saveDir=os.path.join(saveDir,f'{bestMod.model.__class__.__name__}-acc={round(bestMod.acc,5)}-loss={round(bestMod.loss,6)}-max_epochs={config.epochs}-1100'),
    bestMod=bestMod,
    train_log=train_logs,
    config=config
)

save_weights\BinaryClassificationMobileNetV3Large-acc=0.63616-loss=1.794421-max_epochs=20-1100


In [42]:
def get_ROCandAUC(labels,preds_prob):
    fpr, tpr, thresholds = roc_curve([labels], [preds_prob])
    auc_score =  roc_auc_score([labels], [preds_prob])
    return fpr,tpr,auc_score


In [15]:
import torch
from utils import utils
import os

# config=Config(os.path.join(dir,'config.json'))
# model=BestSelector(os.path.join(dir,'metrics.json'))

metrics=train_val.validate_model(
    bestMod.model,
    test_loader,
    config.device
)



In [16]:
metrics

{'accuracy': 0.1512,
 'precision': 0.14666736900060778,
 'recall': 0.15336779395873293,
 'f1': 0.07892765793171844,
 'ap': 0.07496544401993833}

In [ ]:
import json


fpr,tpr,auc_score=test.get_ROCandAUC(
    labels,
    predicted_scores
)
res={
    "fpr":fpr.tolist(),
    "tpr":tpr.tolist(),
    "auc_score":auc_score
}
json.dump(res,open("roc.json",mode='w'))


In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

precision, recall, threshold = precision_recall_curve(
    labels,
    predicted_scores, 
    pos_label=1
    )
res={
    "precision":precision.tolist(),
    "recall":recall.tolist()
}
json.dump(res,open("pr.json",mode='w'))
fig = plt.figure()
plt.plot(precision, recall, label='Logistic')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()

In [ ]:
# fpr,tpr,auc_score=get_ROCandAUC(labels,predicted_scores)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {auc_score}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


In [ ]:
import json


fpr,tpr,auc_score=test.get_ROCandAUC(
    labels,
    predicted_scores
    )
# fpt_train,tpr_train,auc_train=train_logs.
res={
    "fpr":fpr.tolist(),
    "tpr":tpr.tolist(),
    "auc_score":auc_score
}
# json.dump(res,open("roc.json",mode='w'))